In [1]:
import os
import spacy 
import pickle
import random
import nltk
import pandas as pd
import numpy as np
import nltk
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

In [2]:
#!pip install jsonlines

Resume Dataset:

In [92]:
data_1 = pd.read_csv(r"C:\Users\trainee\Downloads\Mohammed Khalaf\UpdatedResumeDataSet.csv")
data_1

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [93]:
data_1['Resume'][0]

'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigatio

In [94]:
nlp = spacy.load("en_core_web_lg")
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Skills Dataset to make the entities from Resume Dataset:

In [95]:
skill_pattern_path = r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\jz_skill_patterns.jsonl'
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)

In [96]:
import re
clean = []
for i in range(data_1.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        str(data_1["Resume"].iloc[i])
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [97]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset

In [98]:
def unique_skills(x):
    return list(set(x))

In [99]:
data_1["Resume_Cleaned"] = clean
data_1["skills"] = data_1["Resume_Cleaned"].str.lower().apply(get_skills)
data_1["skills"] = data_1["skills"].apply(unique_skills)
data_1.head()

,Category,Resume,Resume_Cleaned,skills
0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,"[kafka, bootstrap, analytics, docker, monitori..."
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may 2013 may 2017 b e uit rgp...,"[python, machine learning, ml, github, feature..."
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,"[django, electrical engineering, segmentation,..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,"[swift, segmentation, algorithm, analytics, sp..."
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,"[python, data analysis, data structure, data s..."


In [100]:
clean_data_resume = data_1['Resume_Cleaned']
clean_data_resume.head()

0    skill programming language python panda numpy ...
1    education detail may 2013 may 2017 b e uit rgp...
2    area interest deep learning control system des...
3    skill r python sap hana tableau sap hana sql s...
4    education detail mca ymcaust faridabad haryana...
Name: Resume_Cleaned, dtype: object

In [101]:
clean_data_resume.shape

(962,)

In [13]:
skills = pd.read_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\Dataset.csv')
skills_data = skills['skills']
skills_data.head

<bound method NDFrame.head of 0                 business
1                 software
2       project management
3                marketing
4              engineering
               ...        
5184              security
5185              software
5186                  None
5187              security
5188              security
Name: skills, Length: 5189, dtype: object>

In [14]:
skills_data = pd.DataFrame(skills_data)
skills_data

,skills
0,business
1,software
2,project management
3,marketing
4,engineering
...,...
5184,security
5185,software
5186,None
5187,security


In [32]:
skills_data[0:4]

['asana', 'aspect-oriented programming', 'eureka', 'quantifier elimination']

In [15]:
annot = nlp(data_1['Resume'][2])
for entity in annot.ents:
    print(entity.text, entity.label_)

Deep Learning SKILL
Control System Design, Programming ORG
Python SKILL
Electric Machinery ORG
Web Development, Analytics Technical Activities ORG
Hindustan Aeronautics Limited ORG
Bangalore GPE
4 weeks DATE
Satish PERSON
Mirage 2000 ORG
Technical Skills Programming Matlab ORG
Python SKILL
Java GPE
Python WebFrameWork-Django ORG
Flask FAC
Languages NORP
MIPOWER ORG
Github GPE
Jupyter Notebook WORK_OF_ART
MySQL SKILL
Python SKILL
Software SKILL
Pycharm ORG
Java GPE
Eclipse SKILL
Operating Systems SKILL
Windows SKILL
Ubuntu PRODUCT
Debian-Kali NORP
Linux SKILL
January 2019 DATE
Electrical and Electronics Engineering  Manipal Institute of Technology ORG
January 2015 DATE
January 2013 DATE
DATA SCIENCE SKILL
DATA SCIENCE SKILL
Exprience - Less than 1 year months PERSON
Exprience - Less than 1 year months PERSON
Casino ORG
Macau GPE
segment SKILL
segmentation SKILL
marketing SKILL
business SKILL


In [16]:
skill = []
for i in data_1['skills']:
    for j in i:
        skill.append(j)
print(len(skill))
skill = list(set(skill))
print(skill)

9971
['task management', 'web service', 'logistic regression', 'terminal', 'material', 'tower', 'user profile', 'analytics', 'docker', 'file format', 'specification', 'intellij idea', 'devops', 'source code', 'civil engineering', 'postgresql', 'correctness', 'hbase', 'router', 'linux', 'data flow diagram', 'design', 'interaction', 'computer science', 'mysql', 'mariadb', 'cluster analysis', 'relational database', 'natural language processing', 'predictive coding', 'pycharm', 'text editor', 'collaboration', 'business administration', 'continuous deployment', 'system integration', 'virtual reality', 'routing protocol', 'angularjs', 'zookeeper', 'data access', 'linear regression', 'network interface', 'front end', 'visualization', 'data quality', 'scikit learn', 'microprocessor', 'screen sharing', 'deep learning', 'logstash', 'data analysis', 'functional testing', 'asp net', 'data management', 'network performance', 'data integration', 'robot', 'business process', 'business', 'data model',

In [17]:
dfile = open(r'C:\Users\trainee\Downloads\Mohammed Khalaf\resume_corpus-master\resume_corpus-master\normlized_classes.txt', 'rt', encoding='ISO-8859-1')
data = dfile.readlines()
print(data)

['front end developer (branding specialist):Front_End_Developer\n', 'mis network analyst:Network_Administrator\n', 'designer & front- end developer:Front_End_Developer\n', 'network administrator politécnico 5 septiembre:Network_Administrator\n', 'web developer / .net developer:Web_Developer\n', 'senior project manager - united launch alliance (ula):Project_manager\n', 'product manager/ project manager:Project_manager\n', 'network/firewall engineer:Network_Administrator\n', 'web producer/ web developer:Web_Developer\n', 'database specialist & compliancewire administrator:Database_Administrator\n', 'manager of it infrastructure and operations:Project_manager\n', 'security analyst:Security_Analyst\n', 'software engineer/ systems analyst:Software_Developer,Systems_Administrator\n', 'oracle production dba:Database_Administrator\n', 'software developer / linux server administrator:Software_Developer\n', 'network administrator/ i.t:Network_Administrator\n', 'database administrator / finance s

In [70]:
mydata = []
with open(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\IT Skills.txt', 'rt') as f:
    data = f.readlines(50000)

for i in data:
    clean_line = i[:400].replace(':', '')
    if clean_line != '\n':
        print(clean_line)
        mydata.append(clean_line)
print(mydata)

1Senior Systems Administrator;Systems Administrator IISenior Systems Administrator Brownsville, TX Authorized to work in United States for any employer Work Experience Senior Systems Administrator Trico Products - Brownsville, TX October 2016 to October 2018 Senior Systems Administrator for Trico Products Corporation Technical lead for the Infrastructure Group at the following Locations  N
2Systems AdministratorSystems Administrator Metairie, LA Driven Technologist with a broad range of support skills and a knack for web development. Experienced in object-oriented programming; developing, testing and debugging code; and administering systems and networks. Junior Web developer able to build a Web presence from the ground up. Fast learner, hard worker and team player who is prof
3Systems Administrator;Computer Systems AssistantSystems Administrator Systems Administrator - Nord Gear Corporation Madison, WI Nord Gear Corporation  Deploy and Maintain Windows Server environment across USA, C

In [71]:
# reading skills from json file 
import json

f = open(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\jz_skill_patterns.jsonl', 'r')
skills_data = []
for line in f:
    f_line = json.loads(line)
    dummy = []
    for i in f_line['pattern']:
        dummy.append(list(i.values())[0].lower())
    dummy = ' '.join(dummy)
    skills_data.append(dummy)
    
print(len(skills_data))
print(skills_data)
skills_data = list(set(skills_data))
print(len(skills_data))
print(skills_data)

2129
['.net', ' net', '1password', '3d', '3d reconstruction', 'aboutness', 'abstract data type', 'abstract interpretation', 'abstract machine', 'access control', 'access method', 'access network', 'accounting', 'active appearance model', 'active database', 'active networking', 'active shape model', 'activemq', 'activity recognition', 'actuarial science', 'actuator', 'adaboost', 'adaptive routing', 'adaptive system', 'adder', 'adobe illustrator', 'adobe photoshop', 'advertising', 'aerial photography', 'aeronautics', 'aerospace engineering', 'aerospike', 'agile project management', 'agricultural engineering', 'airflow', 'airtable', 'ajax', 'akamai', 'akka', 'algolia', 'algorithms', 'algorithm', 'algorithm design', 'alpine linux', 'amazon api gateway', 'amazon athena', 'amazon cloudfront', 'amazon cloudwatch', 'amazon cognito', 'amazon dynamodb', 'amazon ebs', 'amazon ec2', 'amazon ec2 container service', 'amazon eks', 'amazon elasticache', 'amazon elasticsearch service', 'amazon emr', 'a

In [72]:
for n in range(len(skills_data)):
    skills_data[n] = re.sub(r'\(|\)', '', skills_data[n])
print(skills_data)



['asana', 'aspect-oriented programming', 'eureka', 'quantifier elimination', 'apache flink', 'objective-c', 'link analysis', 'java build tools', 'sass', 'knowledge-based systems', 'nuxt', 'softlayer', 'command line interface', 'biometrics', 'dyn', 'beta testing / mobile app distribution', 'mongodb hosting', 'spread spectrum', 'aws direct connect', 'web servers', 'load balancer   reverse proxy', 'payments', 'mastodon', 'cloud hosting', 'relational database', 'natural language processing', 'php mvc', 'virtual reality', 'amazon kinesis', 'network interface', 'framer', 'mesh networking', 'real time operating system', 'spatial frequency', 'mobile backend', 'cataloging', 'travelling salesman problem', 'google cloud sql', 'inspec', 'mongodb atlas', 'pico-8', 'data logger', 'raspberry pi', 'namecheap', 'analysis of variance', 'koding', 'elixir', 'point location', 'marathon', 'javascript', 'communications', 'data warehouse', 'context-free language', 'sma*', 'electronic document', 'debian', 'cor

In [73]:
import csv 
Type = 'SKILLS'
with open(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\jobzilla_skills_it.csv', "w", encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Text', 'Type'])
    for job in skills_data:
        csv_writer.writerow([job, Type])

In [74]:
jobzilla_skills_it = pd.read_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\jobzilla_skills_it.csv')
jobzilla_skills_it

,Text,Type
0,asana,SKILLS
1,aspect-oriented programming,SKILLS
2,eureka,SKILLS
3,quantifier elimination,SKILLS
4,apache flink,SKILLS
...,...,...
2105,zeromq,SKILLS
2106,web app builders,SKILLS
2107,bag of words model,SKILLS
2108,password management,SKILLS


In [75]:
f2 = pd.DataFrame(jobzilla_skills_it['Text'])
f2

,Text
0,asana
1,aspect-oriented programming
2,eureka
3,quantifier elimination
4,apache flink
...,...
2105,zeromq
2106,web app builders
2107,bag of words model
2108,password management


In [76]:
myData_Skills = pd.concat([skills_data ,f2])
myData_Skills

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [77]:
myData_Skills.shape

(2110, 2)

In [78]:
myData_Skills.duplicated().sum()

0

In [79]:
myData_Skills.drop_duplicates(inplace=True)
myData_Skills

,Text,Type
0,customer support chat,SKILLS
1,templating languages & extensions,SKILLS
2,multimedia database,SKILLS
3,localhost tools,SKILLS
4,flyway,SKILLS
...,...,...
2105,word sense disambiguation,SKILLS
2106,computational geometry,SKILLS
2107,druid,SKILLS
2108,self-hosted blogging / cms,SKILLS


In [81]:
myData_Skills= pd.read_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\myData_Skills.csv')
myData_Skills

,Text,Type
0,customer support chat,SKILLS
1,templating languages & extensions,SKILLS
2,multimedia database,SKILLS
3,localhost tools,SKILLS
4,flyway,SKILLS
...,...,...
2105,word sense disambiguation,SKILLS
2106,computational geometry,SKILLS
2107,druid,SKILLS
2108,self-hosted blogging / cms,SKILLS


In [82]:
Type = 'SKILLS'
with open(r"C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\myData_Skills.csv", "w", encoding="utf-8") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Text', 'Type'])
    for job in skills_data:
        csv_writer.writerow([job, Type])

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\trainee\\Downloads\\Mohammed Khalaf\\HITS internship\\myData_Skills.csv'

In [83]:
myData_Skills = pd.read_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\myData_Skills.csv')
myData_Skills

,Text,Type
0,customer support chat,SKILLS
1,templating languages & extensions,SKILLS
2,multimedia database,SKILLS
3,localhost tools,SKILLS
4,flyway,SKILLS
...,...,...
2105,word sense disambiguation,SKILLS
2106,computational geometry,SKILLS
2107,druid,SKILLS
2108,self-hosted blogging / cms,SKILLS


In [51]:
dropCells = myData_Skills[(myData_Skills.Text == 'None')].index
dropCells

Int64Index([], dtype='int64')

In [52]:
'''myData_Skills=myData_Skills.drop([  82,  447,  548,  577,  578,  687,  696,  866,  883,  916, 1164,
            1173, 1181, 1182, 1187, 1238, 1295, 1376, 1568, 1655, 1726, 1787,
            1973, 2048, 2312, 2321, 2576, 3245, 3354, 3418, 3493, 4397, 4451,
            4503, 4626, 4627, 4710, 4711, 4712, 4713, 4714, 4715, 4720, 4721,
            4726, 4729, 4730, 4739, 4740, 4742, 4752, 4753, 4754, 4772, 4773,
            4791, 4796, 4805, 4806, 4826, 4827, 4842, 4893, 4900, 4968, 5000,
            5009, 5017, 5027, 5077, 5097, 5111, 5158, 5163, 5181, 5186] , axis=0)'''

'myData_Skills=myData_Skills.drop([  82,  447,  548,  577,  578,  687,  696,  866,  883,  916, 1164,\n            1173, 1181, 1182, 1187, 1238, 1295, 1376, 1568, 1655, 1726, 1787,\n            1973, 2048, 2312, 2321, 2576, 3245, 3354, 3418, 3493, 4397, 4451,\n            4503, 4626, 4627, 4710, 4711, 4712, 4713, 4714, 4715, 4720, 4721,\n            4726, 4729, 4730, 4739, 4740, 4742, 4752, 4753, 4754, 4772, 4773,\n            4791, 4796, 4805, 4806, 4826, 4827, 4842, 4893, 4900, 4968, 5000,\n            5009, 5017, 5027, 5077, 5097, 5111, 5158, 5163, 5181, 5186] , axis=0)'

In [118]:
myData_Skills

,Text,Type
0,customer support chat,SKILLS
1,templating languages & extensions,SKILLS
2,multimedia database,SKILLS
3,localhost tools,SKILLS
4,flyway,SKILLS
...,...,...
2105,word sense disambiguation,SKILLS
2106,computational geometry,SKILLS
2107,druid,SKILLS
2108,self-hosted blogging / cms,SKILLS


In [102]:
data_resume = pd.read_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\clean_resume_skills-category.csv')
data_resume.head()

,Unnamed: 0,Clean_Resume,skills,Category
0,0,skill programming language python panda numpy ...,"['mysql', 'data science', 'decision tree', 'co...",Data Science
1,1,education detail may 2013 may 2017 b e uit rgp...,"['github', 'feature selection', 'python', 'mac...",Data Science
2,2,area interest deep learning control system des...,"['mysql', 'data science', 'django', 'python', ...",Data Science
3,3,skill r python sap hana tableau sap hana sql s...,"['data science', 'swift', 'decision tree', 'py...",Data Science
4,4,education detail mca ymcaust faridabad haryana...,"['data analysis', 'python', 'database']",Data Science


In [103]:
clean_data_resume = data_resume['Clean_Resume']
clean_data_resume.head()

0    skill programming language python panda numpy ...
1    education detail may 2013 may 2017 b e uit rgp...
2    area interest deep learning control system des...
3    skill r python sap hana tableau sap hana sql s...
4    education detail mca ymcaust faridabad haryana...
Name: Clean_Resume, dtype: object

In [104]:
skills = pd.read_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\HITS internship\mySkills.csv')
skills.head()

,Unnamed: 0,0
0,0,serverless
1,1,very large database
2,2,task management
3,3,adaptive system
4,4,mixed model


In [105]:
skills = skills['0']
skills.shape

(9722,)

In [106]:
import re
for n, i in enumerate(skills):
    dummy = re.sub('\'$|\'(?= )|\"|\/ |\*', '', i) # remove any ' " * / 
    dummy = re.sub(r'\/', ' ', dummy) # remove and two words sperated with / char with space like A/B to AB
    dummy = re.sub(r'  |^ ', '', dummy) # remove more that one white space
    if '+' in i:
        dummy = dummy.replace('+', '\+') # change ++ to \+\+
    skills[n] = dummy 
skills.head()

0             serverless
1    very large database
2        task management
3        adaptive system
4            mixed model
Name: 0, dtype: object

In [107]:
max_len = 0
for i in clean_data_resume:
    val_len = len(i)
    if val_len > max_len:
        max_len = val_len
        text = i
print(max_len, text, sep='\n') 

11542
education detail august 2000 b e electronics pune maharashtra pune university operation manager operation manager delta control dubai fzco skill detail company detail company delta control dubai fzco description heading pune branch deltannex integrator pvt ltd aug 17 till date designation operation manager 1st employee 1 reporting gm operation 2 review sow rfq assist proposal team engineering man hour project schedule organization chart meeting client pre sale support 3 review sow rfq client purchase order contract term condition 4 project execution plan discussion client consultant 5 kick meeting ad hoc meeting client 6 project evaluation term outlay profit 7 ensuring appropriate project management framework 8 management contractual commercial issue related project 9 monitoring reviewing reporting project progress 10 coordinating fostering teamwork prioritization team activity 11 monitor project budget 12 apply quality management system process 13 liaison reporting client 14 mon

In [109]:
data = data_resume['Clean_Resume'][0]
data

'skill programming language python panda numpy scipy scikit learn matplotlib sql java javascript jquery machine learning regression svm na bayes knn random forest decision tree boosting technique cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca neural net database visualization mysql sqlserver cassandra hbase elasticsearch d3 j dc j plotly kibana matplotlib ggplot tableau others regular expression html cs angular 6 logstash kafka python flask git docker computer vision open cv understanding deep learning education detail data science assurance associate data science assurance associate ernst young llp skill detail javascript exprience 24 month jquery exprience 24 month python exprience 24 monthscompany detail company ernst young llp description fraud investigation dispute service assurance technology assisted review tar technology assisted review assist accelerating review process run analytics generate 

In [120]:
%%time
import re
from tqdm import tqdm

label = 'SKILLS'
output_data = []
for sentence in tqdm(data_1["Resume_Cleaned"]):
    data_entities = {'entities': []}
    for skill in skills:
        for i in re.finditer(r'\b{skill}\b', sentence):
            data_entities['entities'].append((i.start(), i.end(), label))

output_data.append([sentence, data_entities])

100%|██████████| 962/962 [07:39<00:00,  2.09it/s]

Wall time: 7min 39s


In [124]:
output_data

[['skill set o window xp 7 8 8 1 10 database mysql sql server 2005 2008 2012 language core java web technology html cs testing manual testing database testing bug tracking reporting end user handling education detail january 2016 mc pune maharashtra p v p college pravaranagar january 2011 hsc k junior college january 2009 ssc k mahavidyalya jr testing engineer jr testing engineer tech innovation ltd skill detail testing exprience 24 month window xp exprience 24 month cs exprience 6 month database exprience 6 month database testing exprience 6 monthscompany detail company tech innovation ltd description responsibility analyzing testing requirement preparing test plan test scenario preparing test case executing test case tracking bug coordinating developer order fix preparing test summary report reporting quality manager attending meeting project project name 1 web based time attendance application environment window 7 vista window xp window 8 window 10 client mccia sapphire park bramha 

In [132]:
def auto_annotate(data, skills, label='SKILLS'):
    output_data = []
    for record in tqdm(data):
        data_entities = {'entities': []}
        for skill in skills:
            for i in re.finditer(r'\b{skill}\b', record):
                data_entities['entities'].append((i.start(), i.end(), label))
    output_data.append([record, data_entities])
    return output_data


In [133]:
#output_data

In [134]:
output_data = auto_annotate(data_1["Resume_Cleaned"][:10], myData_Skills['Text'], label='SKILLS')
print(output_data)
for i in output_data:
    print(i[1])

  0%|          | 0/10 [00:00<?, ?it/s]


error: multiple repeat at position 3

In [ ]:
def train_dev_split(all_data, percent=0.2):
  # shuffle the data
    data = all_data.copy()
    random.shuffle(data)

  # split the data to train, dev sets 
    split_len = int(len(data) * (1-percent) )
    train_data = data[:split_len]
    dev_data = data[split_len:]
    return train_data, dev_data

In [ ]:
def make_spacy_file(data, path):
    nlp = spacy.blank('en') # load a new spacy model
    db = DocBin() # create a DocBin object
    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot['entities']: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode='contract')
            if span is None:
                print('Skipping entity')
                # print(span, start, end, label, text[start:end])
            else:
                ents.append(span)
            try:
                filtered = filter_spans(ents)
                doc.ents = filtered # label the text with the ents
                db.add(doc)
            except:
            print(text, annot)
    db.to_disk(f'{path}'.spacy') # save the docbin object

In [ ]:
train_data, test_data = train_dev_split(output_data, percent=0.15)

In [ ]:
make_spacy_file(train_data, train_path)
make_spacy_file(test_data, test_path)